<span style="font-size:36px"><b>Preprocess Bibleis</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [2]:
import os
import re
import glob
import json

import numpy as np
import pandas as pd

# Preprocess Transcription for Alignment

Preprocess each chapter transcription by:
1. Splitting each sentence divided by '.'
2. Removing any character except `"a-z"`, `"."`, `","`, `"<space>"`,
3. Write each of chapter verse to a `.txt` file.

In [3]:
df = pd.read_csv("../../dataset/processed/bibleis_trimmed.csv")

In [4]:
df[df['audio_title'] == 'INDASV_1CH_1.mp3'].values

array([['https://live.bible.is/bible/INDASV/1CH/1?audio_type=audio',
        "Adam, Set, Enos,\n\nKenan, Mahalaleel, Yared,\n\nHenokh, Metusalah, Lamekh,\n\nNuh, Sem, Ham dan Yafet.\n\nKeturunan Yafet ialah Gomer, Magog, Madai, Yawan, Tubal, Mesekh dan Tiras.\n\nKeturunan Gomer ialah Askenas, Difat dan Togarma.\n\nKeturunan Yawan ialah Elisa, Tarsis, orang Kitim dan orang Rodanim.\n\nKeturunan Ham ialah Kush, Misraim, Put dan Kanaan.\n\nKeturunan Kush ialah Seba, Hawila, Sabta, Raema dan Sabtekha; keturunan Raema ialah Syeba dan Dedan.\n\nKush memperanakkan Nimrod; dialah orang yang mula-mula sekali berkuasa di bumi.\n\nMisraim memperanakkan orang Ludim, orang Anamim, orang Lehabim, orang Naftuhim,\n\norang Patrusim, orang Kasluhim — dari mereka inilah berasal orang Filistin — dan orang Kaftorim.\n\nKanaan memperanakkan Sidon, anak sulungnya dan Het,\n\nserta orang Yebusi, orang Amori, orang Girgasi,\n\norang Hewi, orang Arki, orang Sini,\n\norang Arwadi, orang Semari dan orang Hamati.

In [5]:
def clean_str(x):
    return re.sub(r'[^a-zA-z.,\n ]', '', x)

In [6]:
tmp = [x.replace('\n\n', ' ').lower() for x in df['chapter_string']]
tmp = [x.replace('. ', '.\n') for x in tmp]
tmp = [re.sub(r'[-]', ' ', x) for x in tmp]
tmp = [clean_str(x) for x in tmp]

Store the result back in the dataframe and see example of cleaned transcription.

In [7]:
df['chapter_string'] = tmp

In [8]:
df.sample(1)['chapter_string'].values

array(['ah, betapa tuhan menyelubungi puteri sion dengan awan dalam murka nya keagungan israel dilemparkan nya dari langit ke bumi.\ntak diingat nya akan tumpuan kaki nya tatkala ia murka.\ntanpa belas kasihan tuhan memusnahkan segala ladang yakub.\nia menghancurkan dalam amarah nya benteng benteng puteri yehuda.\nia mencampakkan ke bumi dan mencemarkan kerajaan dan pemimpin pemimpinnya.\ndalam murka yang menyala nyala ia mematahkan segala tanduk israel, menarik kembali tangan kanan nya pada waktu si seteru mendekat, membakar yakub laksana api yang menyala nyala, yang menjilat ke sekeliling.\nia membidikkan panah nya seperti seorang seteru dengan mengacungkan tangan kanan nya seperti seorang lawan membunuh segala yang menyenangkan mata dalam kemah puteri sion, memuntahkan geram nya seperti api.\ntuhan menjadi seperti seorang seteru ia menghancurkan israel, meremukkan segala purinya, mempuingkan benteng bentengnya, memperbanyak susah dan kesah pada puteri yehuda.\nia melanda kemah nya s

Write the cleaned transcription into `.txt` files.

In [9]:
# for x in df.values:
#     with open(x[2][:-4] + '.txt', 'w', encoding='utf-8') as f:
#         f.writelines(x[1])

# Preprocess Audio & Text After Alignment

Given aligned `.json` from aeneas output, split each audio sentence into its own `.mp3` and `.txt` files.

In [10]:
from gurih.data.splitter import AeneasSplitter

ModuleNotFoundError: No module named 'gurih.data.splitter'

In [4]:
input_dir = '../../dataset/processed/bibleis_trimmed/'
output_dir = '../../dataset/processed/bibleis_trimmed_splitted/'
splitter = AeneasSplitter(input_dir=input_dir, output_dir=output_dir)

In [5]:
mp3s = glob.glob(input_dir+"*.json")
mp3s = [os.path.basename(path) for path in mp3s]

In [7]:
# mp3s[126:]

In [8]:
for mp3 in mp3s[126:]:
    fragments = splitter.load(mp3)
    splitter.split_and_write(fragments)

In [2]:
from pydub import AudioSegment
from pydub.utils import mediainfo

In [3]:
aligned_json_files = list(glob.glob("bibleis_trimmed/*.json"))
print("Found " + str(len(aligned_json_files))+ " json files")

Found 1189 json files


In [5]:
filenames = aligned_json_files[5]
with open(filenames, 'r') as f:
    data = ''.join(f.readlines()) # read the files as string
    
fragments = json.loads(data)['fragments'] # convert to dict, all data is stored in 'fragments' key

In [6]:
fragments[:2]

[{'begin': '0.000',
  'children': [],
  'end': '19.400',
  'id': 'f000001',
  'language': 'ind',
  'lines': ['hiram, raja negeri tirus, mengirim utusan kepada daud dan kayu aras, tukang tukang batu dan tukang tukang kayu untuk mendirikan sebuah istana baginya.']},
 {'begin': '19.400',
  'children': [],
  'end': '35.800',
  'id': 'f000002',
  'language': 'ind',
  'lines': ['lalu tahulah daud, bahwa tuhan telah menegakkan dia sebagai raja atas israel, sebab martabat pemerintahannya terangkat tinggi oleh karena israel umat nya.']}]

In [7]:
fragment = fragments[0]

In [8]:
t_begin = float(fragment['begin']) * 1000 # miliseconds
t_end = float(fragment['end']) * 1000 # miliseconds

out_filename = filenames[:-5] + '_' + fragment['id'] + ".mp3"
new_audio = AudioSegment.from_mp3(filenames[:-5]+".mp3")
new_audio = new_audio[t_begin:t_end]
new_audio.export(out_filename, format="mp3")

<_io.BufferedRandom name='bibleis_trimmed\\INDASV_1CH_14_f000001.mp3'>

Test.

In [13]:
input_info

{'index': '1',
 'codec_name': 'mjpeg',
 'codec_long_name': 'Motion JPEG',
 'profile': '192',
 'codec_type': 'video',
 'codec_time_base': '0/1',
 'codec_tag_string': '[0][0][0][0]',
 'codec_tag': '0x0000',
 'sample_fmt': 'fltp',
 'sample_rate': '24000',
 'channels': '2',
 'channel_layout': 'stereo',
 'bits_per_sample': '0',
 'id': 'N/A',
 'r_frame_rate': '90000/1',
 'avg_frame_rate': '0/0',
 'time_base': '1/90000',
 'start_pts': 'N/A',
 'start_time': '0.000000',
 'duration_ts': '21679200',
 'duration': '240.880000',
 'bit_rate': '64378',
 'max_bit_rate': 'N/A',
 'bits_per_raw_sample': '8',
 'nb_frames': 'N/A',
 'nb_read_frames': 'N/A',
 'nb_read_packets': 'N/A',
 'DISPOSITION': {'default': '0',
  'dub': '0',
  'original': '0',
  'comment': '0',
  'lyrics': '0',
  'karaoke': '0',
  'forced': '0',
  'hearing_impaired': '0',
  'visual_impaired': '0',
  'clean_effects': '0',
  'attached_pic': '1',
  'timed_thumbnails': '0'},
 'width': '300',
 'height': '300',
 'coded_width': '300',
 'coded_

In [12]:
test_audio = AudioSegment.from_mp3(out_filename)
input_info = mediainfo(filenames[:-5]+".mp3")
output_info  = mediainfo(out_filename)

In [27]:
assert test_audio.duration_seconds == (t_end - t_begin) / 1000, "Output input length doesn't match"
assert input_info['sample_rate'] == output_info['sample_rate'], "Output input sample rate doesn't match"